# NLP

##

### Separación de subconjuntos de entrenamiento y testeo
Generar X_train, X_test, y_train e y_test de nuevo, ya que las modificamos anteriormente con el count vectorizer:

In [ ]:
X_train, X_test, y_traim, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

## Vectorización TF-IDF

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


Ahora, importar tfidf vectorizer y aplicarlo sobre X_train y X_test

In [ ]:
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(X_train)

TfidfVectorizer()

In [ ]:
X_train = tfidf_vec.transform(X_train)

In [ ]:
X_test = tfidf_vec.transform(X_test)

NameError: name 'tfidf_vec' is not defined

## MLP

Dado que el resultado de la vectorización clasica, nos da un dataframe de alta dimensionalidad y relaciones muy complejas y ocultas entre variables, es una excelente oportunidad para comprobar como se comporta un perceptron multicapa.

In [ ]:
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
import matplotlib.pyplot as plt

### Preprocesamiento adicional para nuestro MLP

Como vimos, algo que diferencia a las redes neuronales de algortimos clasicos. de ML es la no necesidad de feature engineer, pero entiendo esta como la creación de columnas derivadas en varse aciertos patrones; el preprocesamiento y limpieza seguiran siendo super necesarios para que la red pueda aprender.

Nuestro MLP necesitara la columna target en formato numerico, por lo que previamente la binarizaremos.

In [ ]:
lb = LabelBinarizer()
lb.fit(y_train)

y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

X_train debe estar en formato denso (no sparse), lo convertiremos a tal formato:

In [ ]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 115294 stored elements and shape (1400, 17202)>

In [ ]:
X_train = X_train.toarray()
X_train = X_train.astype('float32')

### Definiendo las diferentes capas del MLP:

- Capa de entrada: Tendra x caracteristicas de entrada igual a la cantidad. de features / caracteristicas del dataset vectorizado (cantidad de palabras en el vocavulario)
- Capas ocultas: Dado la complegidad y dimension del conjunton de datos, estara compuesto por tres capas ocultas de x neuronas cada una.
- Capa de salida: Como es un problema. declasificación binario, se utilizara una sola neurona que predicira la probabilidad de la clase positiva.

Capas de entrada

In [ ]:
capa_entrada = Input(shape=(X_train.shape[1],))

Capas ocultas

In [ ]:
cantidad_neuronas_capa = int((X_train.shape[1] + len(np.unique(y_train)) / 2))

In [ ]:
capa_oculta_uno = Dense(units=cantidad_neuronas_capa, activation='relu')

In [ ]:
capa_oculta_dos = Dense(units=cantidad_neuronas_capa, activation='relu')

In [ ]:
capa_oculta_tres = Dense(units=cantidad_neuronas_capa, activation='relu')

In [ ]:
capa_oculta_cuatro = Dense(units=cantidad_neuronas_capa, activation='relu')

In [ ]:
capa_oculta_cinco = Dense(units=cantidad_neuronas_capa, activation='relu')

Capas de salida

In [ ]:
capa_salida = Dense(units=1, activation='sigmoid')

### Orquestación de las neuronas dentro del modelo de red sequencial:

Dado que el perceptron es un modelo de red neuronal donde todas las neuronas estan **secuencialmente** y **densamente** o totalmente conectadas con la capa siguiente, etonces utilizaremos el modelo `Sequential` para conectar las capas densas una despues de la otra.

In [ ]:
mlp_model = Sequential(layers= [
    capa_entrada,
    capa_oculta_uno,
    capa_oculta_dos,
    capa_oculta_tres,
    capa_oculta_cuatro,
    capa_oculta_cinco,
    capa_salida
]
)

In [ ]:
mlp_model.compile(optimizer=Adam(0.01), loss='BinaryCrossentropy')

Entrenamiento

In [ ]:
historial = mlp_model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=700)

Epoch 1/10


In [ ]:
y_pred = mlp_model.predict(X_test)

In [ ]:
y_pred = (y_pred >= 0.5).astype(int)

In [ ]:
print(classification_report(y_pred=y_pred, y_true=y_test))

In [ ]:
historial.history.keys()

In [ ]:
error_train = historial.history['loss']
error_val = historial.history['val_loss']

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(error_train, label='train loss', color='r')
plt.plot(error_val, label='val loss', color='b')
plt.legend()
plt.show()